In [1]:
%load_ext autoreload
%autoreload 2

# Feature extraction

In [2]:
import pandas as pd
import numpy as np
import feature_extraction
import time
from os import path, mkdir

In [3]:
data_path = "../data"
feat_path = "../features"

tesseract_path = r"%localappdata%\Tesseract-OCR\tesseract"

## Load dataset

In [4]:
dataset = "clickbait17-train-170331"

In [5]:
# Load instances
df = pd.read_json(path.join(data_path, dataset, "instances.jsonl"), lines=True, encoding='utf8')
df.set_index("id", inplace=True)
df_raw = df.copy()

# Load truth judgments
df_truth = pd.read_json(path.join(data_path, dataset, "truth.jsonl"), lines=True, encoding='utf8')
df_truth.set_index("id", inplace=True)
df_truth_raw = df_truth.copy()

# Add truth judgments to df
df = df.join(df_truth[["truthClass"]]) # also available: truthClass, truthJudgments, truthMean, truthMedian, truthMode

In [6]:
with pd.option_context('max_colwidth', 500):
    display(df.head(10))

,postMedia,postText,postTimestamp,targetCaptions,targetDescription,targetKeywords,targetParagraphs,targetTitle,truthClass
id,,,,,,,,,
608310377143799808,[],[Apple's iOS 9 'App thinning' feature will give your phone's storage a boost],Tue Jun 09 16:31:10 +0000 2015,"['App thinning' will be supported on Apple's iOS 9 and later models. It ensures apps use the lowest amount of storage space on a device by only downloading the parts it needs to run on individual handsets. It 'slices' the app into 'app variants' that only need to access the specific files on that specific device, 'App thinning' will be supported on Apple's iOS 9 and later models. It ensures apps use the lowest amount of storage space on a device by only downloading the parts it needs to run ...",'App thinning' will be supported on Apple's iOS 9 and later models. It ensures apps use the lowest amount of storage space by 'slicing' it to work on individual handsets (illustrated).,"Apple,gives,gigabytes,iOS,9,app,thinning,feature,finally,phone,s,storage,boost","[Paying for a 64GB phone only to discover that this is significantly reduced by system files and bloatware is the bane of many smartphone owner's lives. , And the issue became so serious earlier this year that some Apple users even sued the company over it. , But with the launch of iOS 9, Apple is hoping to address storage concerns by introducing a feature known as 'app thinning.', It has been explained on the watchOS Developer Library site and is aimed at developers looking to optimise thei...",Apple gives back gigabytes: iOS 9 'app thinning' feature will finally give your phone's storage a boost,no-clickbait
609297109095972864,[media/609297109095972864.jpg],"[RT @kenbrown12: Emerging market investors are doing their best Monty Pythons--""Run away, run away""]",Fri Jun 12 09:52:05 +0000 2015,"[Stocks Fall as Investors Watch Central Banks, Do Traders See Data Before Release? Markets Say Yes, Be Careful: Stock Volatility is Hiding, Not Hibernating, Singapore Bans Ex-Goldman Banker Linked to 1MDB From Working in Financial Sector, Demanding and Direct: HSBC’s New Chairman Mark Tucker, Saudi King’s Asia Expedition Enters Crucial Phase, Puerto Rico Board Approves Fiscal Road Map, CFPB, Justice Department Poised to Square Off in Court, Raphael Bostic to Lead Atlanta Fed, Blackstone Sell...","Global investors have yanked $9.3 billion from stocks in developing countries in the week to Wednesday, the most since the depths of the 2008 global financial crisis.","emerging market,emerging markets,em flows,em inflow,em outflow,equity markets,money,forex markets,commodity,financial market news","[Emerging markets are out of favor., Global investors have yanked $9.3 billion from stocks in developing countries in the week to Wednesday, the most since the depths of the global financial crisis in 2008. Asia has been particularly vulnerable with $7.9 billion pulled out of the region’s equity markets, the most in almost 15 years, according to data provider EPFR Global., Financial markets in emerging markets have been grinding...]",Emerging Markets Suffer Largest Outflow in Seven Years,no-clickbait
609504474621612032,[],"[U.S. Soccer should start answering tough questions about Hope Solo, @eric_adelson writes.]",Fri Jun 12 23:36:05 +0000 2015,"[US to vote for Ali in FIFA election and not Blatter, US to vote for Ali in FIFA election and not Blatter, FILE - This Oct. 10, 2014, file photo shows Sunil Gulati, president of the United States Soccer Federation, during a press conference in Bristol, Conn. The United States says it will vote for Jordan's Prince Ali bin Al-Hussein for FIFA president Friday, May 29, 2015 and not for incumbent Sepp Blatter. (AP Photo/Elise Amendola, File)]",A U.S. Senator's scathing letter questioned U.S. Soccer's inadequate handling of Solo's domestic violence charges. It's time for Sunil Gulati to respond.,,"[WINNIPEG, Manitoba – The bubble U.S. Soccer is putting around Hope Solo isn't working to calm anyone's 

## Extract features

In [13]:
run_id = time.strftime("%Y-%m-%d_%H%M%S")
print("Run ID: %s" % run_id)

Run ID: 2019-04-03_204436


In [14]:
fe = feature_extraction.FeatureExtractor(path.join(data_path, dataset), tesseract_path)
fe.set_df(df)  # <-- 5 items for testing

In [15]:
labels, features = fe.extract_features()

In [16]:
display(features[:10])
labels[:10]

,numChars_post_title,numChars_post_image,numChars_article_keywords,numChars_article_desc,numChars_article_title,numChars_article_paragraphs,numWords_post_title,numWords_post_image,numWords_article_keywords,numWords_article_desc,...,diffStopWords_post_title_post_image,diffStopWords_post_title_article_keywords,diffStopWords_post_title_article_descr,diffStopWords_post_title_article_title,diffStopWords_post_image_article_keywords,diffStopWords_post_image_article_descr,diffStopWords_post_image_article_title,diffStopWords_article_keywords_article_descr,diffStopWords_article_keywords_article_title,diffStopWords_article_descr_article_title
id,,,,,,,,,,,,,,,,,,,,,
608310377143799808,63.0,0.0,78.0,154.0,87.0,99.323529,15.0,0.0,12.0,32.0,...,-1.0,7.0,21.0,12.0,-1.0,-1.0,-1.0,23.0,14.0,5.0
609297109095972864,85.0,200.0,120.0,140.0,47.0,121.333333,15.0,37.0,19.0,27.0,...,22.0,15.0,14.0,5.0,15.0,14.0,5.0,-1.0,-1.0,3.0
609504474621612032,78.0,0.0,0.0,132.0,57.0,212.687500,12.0,0.0,0.0,26.0,...,-1.0,-1.0,14.0,7.0,-1.0,-1.0,-1.0,-1.0,-1.0,6.0
609748367049105408,52.0,0.0,114.0,98.0,52.0,218.560976,11.0,0.0,18.0,18.0,...,-1.0,15.0,15.0,8.0,-1.0,-1.0,-1.0,15.0,8.0,5.0
608688782821453824,76.0,0.0,58.0,103.0,29.0,227.619048,16.0,0.0,11.0,23.0,...,-1.0,3.0,12.0,0.0,-1.0,-1.0,-1.0,13.0,1.0,5.0
609551038983475200,54.0,60.0,0.0,0.0,55.0,142.380952,11.0,11.0,0.0,0.0,...,1.0,-1.0,-1.0,4.0,-1.0,-1.0,6.0,-1.0,-1.0,-1.0
609447408955719680,77.0,0.0,57.0,123.0,67.0,157.709677,17.0,0.0,10.0,26.0,...,-1.0,1.0,14.0,2.0,-1.0,-1.0,-1.0,-1.0,-1.0,3.0
609027430624288768,57.0,0.0,0.0,143.0,56.0,125.800000,11.0,0.0,0.0,29.0,...,-1.0,-1.0,21.0,5.0,-1.0,-1.0,-1.0,-1.0,-1.0,3.0
608229011572068352,70.0,0.0,159.0,115.0,52.0,272.000000,15.0,0.0,20.0,28.0,...,-1.0,15.0,18.0,9.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0


array([0, 0, 1, 0, 1, 1, 0, 0, 0, 0], dtype=int64)

## Write to file

In [17]:
output_path = path.join(feat_path, run_id)

# Make dir for this run
try:
    mkdir(output_path)
except FileExistsError:
    pass

In [18]:
# Write features to file
features.to_pickle(path.join(output_path, 'features.pkl'))

# Write labels to file
np.save(path.join(output_path, 'labels.npy'), labels, allow_pickle=True)

# Read back with:
# features = pd.read_pickle(path.join(output_path, 'features.pkl'))
# labels = np.load(path.join(output_path, 'labels.npy'))